In [2]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel

In [3]:
cluster = Cluster(["127.0.0.1"])
session = cluster.connect()
session.set_keyspace("ex1")

In [4]:
# Depricated
# session.default_consistency_level = ConsistencyLevel.ONE
player = "pcmanus"

results = session.execute("SELECT * FROM scores WHERE user= %(plr)s limit 10", {"plr": player})
for row in results:
    print(f"Zawodnik {row.user} uzyskal {row.score} w roku {row.year}")
    
stmt = SimpleStatement("SELECT * FROM scores WHERE user=%s",
                      consistency_level=ConsistencyLevel.QUORUM)
results = session.execute(stmt, [player])
for row in results:
    print(row)

Zawodnik pcmanus uzyskal 2000 w roku 2015
Zawodnik pcmanus uzyskal 3000 w roku 2020
Zawodnik pcmanus uzyskal 3000 w roku 2020
Row(user='pcmanus', game='Coup', year=2015, address=OrderedMapSerializedKey([('home', 'Warsaw'), ('work', 'Piaseczno')]), detailed_scores=OrderedMapSerializedKey([('game1', [1, 2, 3, 4]), ('game2', [40, 50, 60])]), detailed_scores3=OrderedMapSerializedKey([('game1', SortedSet([1, 2, 3, 4])), ('game2', SortedSet([40, 50, 60]))]), score=2000)
Row(user='pcmanus', game='Coup', year=2020, address=None, detailed_scores=None, detailed_scores3=None, score=3000)
Row(user='pcmanus', game='Coup2020', year=2020, address=None, detailed_scores=None, detailed_scores3=None, score=3000)


In [5]:
stmt = SimpleStatement("INSERT INTO scores (user, game, year, score) values (%s, %s, %s, %s)")
result = session.execute(stmt, ["pcmanus", "Coup", 2020, 3000])

In [38]:
for row in result:
    print(row)

In [40]:
session.execute("ALTER TABLE scores ADD address map<text, text>")
address = {
    'home': 'Warsaw',
    'work': 'Piaseczno'
}
session.execute("UPDATE scores SET address = %s WHERE user='pcmanus' AND game='Coup' AND year=2015", [address])

results = session.execute("SELECT address FROM scores WHERE user='pcmanus'")
for row in results:
    if row.address is None:
        continue
    print(row.address['work'])

Piaseczno


In [49]:
# session.execute("alter table scores drop detailed_scores")
session.execute("ALTER TABLE scores ADD detailed_scores map<text, frozen<list<int>>>")
detailed_scores = {
    'game1': [1,2,3,4],
    'game2': [40,50,60]
}
session.execute("UPDATE scores SET detailed_scores = %s WHERE user='pcmanus' AND game='Coup' AND year=2015", [detailed_scores])

results = session.execute("SELECT detailed_scores FROM scores WHERE user='pcmanus'")
for row in results:
    if row.detailed_scores is None:
        continue
    print(row.detailed_scores)

{'game1': [1, 2, 3, 4], 'game2': [40, 50, 60]}


In [58]:
# session.execute("alter table scores drop detailed_scores")
session.execute("ALTER TABLE scores ADD detailed_scores3 map<text, frozen<set<int>>>")
detailed_scores = {
    'game1': {1,2,3,4, 4},
    'game2': {40,50,60, 60}
}
session.execute("UPDATE scores SET detailed_scores3 = %s WHERE user='pcmanus' AND game='Coup' AND year=2015", [detailed_scores])

results = session.execute("SELECT detailed_scores3 FROM scores WHERE user='pcmanus'")
for row in results:
    if row.detailed_scores3 is None:
        continue
    print(row.detailed_scores3)

{'game1': SortedSet([1, 2, 3, 4]), 'game2': SortedSet([40, 50, 60])}


In [9]:
arg = ("pcmanus", "Coup", 2015, 6000)
result = session.execute("INSERT INTO scores (user, game, year, score) VALUES (%s, %s, %s, %s) IF NOT EXISTS", arg)
for row in result:
    print(row)

Row(applied=False, user='pcmanus', game='Coup', year=2015, address=OrderedMapSerializedKey([('home', 'Warsaw'), ('work', 'Piaseczno')]), detailed_scores=OrderedMapSerializedKey([('game1', [1, 2, 3, 4]), ('game2', [40, 50, 60])]), detailed_scores3=OrderedMapSerializedKey([('game1', SortedSet([1, 2, 3, 4])), ('game2', SortedSet([40, 50, 60]))]), score=3000)


In [11]:
arg = ("pcmanus", "Coup", 2015, 6000)
result = session.execute("INSERT INTO scores (user, game, year, score) VALUES (%s, %s, %s, %s) IF NOT EXISTS", arg)
for row in result:
    print(row.applied)
    print(row)

False
Row(applied=False, user='pcmanus', game='Coup', year=2015, address=OrderedMapSerializedKey([('home', 'Warsaw'), ('work', 'Piaseczno')]), detailed_scores=OrderedMapSerializedKey([('game1', [1, 2, 3, 4]), ('game2', [40, 50, 60])]), detailed_scores3=OrderedMapSerializedKey([('game1', SortedSet([1, 2, 3, 4])), ('game2', SortedSet([40, 50, 60]))]), score=3000)


In [14]:
arg = ("pcmanus", "Coup", 2016, 6000)
result = session.execute("INSERT INTO scores (user, game, year, score) VALUES (%s, %s, %s, %s) IF NOT EXISTS", arg)
for row in result:
    print("INSERT", row.applied)
    print("INSERT", row)

INSERT False
INSERT Row(applied=False, user='pcmanus', game='Coup', year=2016, address=None, detailed_scores=None, detailed_scores3=None, score=6000)


In [13]:
stmt = SimpleStatement("SELECT * FROM scores WHERE user=%s")
results = session.execute(stmt, ["pcmanus"])
for row in results:
    print(row)

Row(user='pcmanus', game='Coup', year=2015, address=OrderedMapSerializedKey([('home', 'Warsaw'), ('work', 'Piaseczno')]), detailed_scores=OrderedMapSerializedKey([('game1', [1, 2, 3, 4]), ('game2', [40, 50, 60])]), detailed_scores3=OrderedMapSerializedKey([('game1', SortedSet([1, 2, 3, 4])), ('game2', SortedSet([40, 50, 60]))]), score=3000)
Row(user='pcmanus', game='Coup', year=2016, address=None, detailed_scores=None, detailed_scores3=None, score=6000)
Row(user='pcmanus', game='Coup', year=2020, address=None, detailed_scores=None, detailed_scores3=None, score=3000)
Row(user='pcmanus', game='Coup2020', year=2020, address=None, detailed_scores=None, detailed_scores3=None, score=3000)


In [19]:
arg = ("pcmanus", "Coup", 2016, 6000)
result = session.execute("INSERT INTO scores (user, game, year, score) VALUES (%s, %s, %s, %s) IF NOT EXISTS", arg)
for row in result:
    print("INSERT", row.applied)
    print("INSERT", row)

    
arg = ("pcmanus", "Coup", 2016)
result = session.execute("DELETE FROM scores WHERE user=%s AND game=%s AND year=%s IF EXISTS", arg)
for row in result:
    print("DELETE", row.applied)
    print("DELETE", row)

INSERT True
INSERT Row(applied=True)
DELETE True
DELETE Row(applied=True)


**Wywołania asynchroniczne**

In [20]:
players = [
    'pcmanus', 'tjake', 'yukim', 'jbellis'
]
for player in players:
    result = session.execute("SELECT * FROM scores WHERE user=%s", [player])
    for row in result:
        print(row)

Row(user='pcmanus', game='Coup', year=2015, address=OrderedMapSerializedKey([('home', 'Warsaw'), ('work', 'Piaseczno')]), detailed_scores=OrderedMapSerializedKey([('game1', [1, 2, 3, 4]), ('game2', [40, 50, 60])]), detailed_scores3=OrderedMapSerializedKey([('game1', SortedSet([1, 2, 3, 4])), ('game2', SortedSet([40, 50, 60]))]), score=3000)
Row(user='pcmanus', game='Coup', year=2020, address=None, detailed_scores=None, detailed_scores3=None, score=3000)
Row(user='pcmanus', game='Coup2020', year=2020, address=None, detailed_scores=None, detailed_scores3=None, score=3000)
Row(user='tjake', game='Coup', year=2015, address=None, detailed_scores=None, detailed_scores3=None, score=1000)
Row(user='yukim', game='Coup', year=2015, address=None, detailed_scores=None, detailed_scores3=None, score=2250)
Row(user='jbellis', game='Coup', year=2015, address=None, detailed_scores=None, detailed_scores3=None, score=1750)


In [27]:
from time import time

players = [
    'pcmanus', 'tjake', 'yukim', 'jbellis'
]
start = time()
for player in players:
    result = session.execute("SELECT * FROM scores WHERE user=%s", [player])
    for row in result:
        print(row)
took = time() - start
print("TOOK", took * 1000, "ms")

Row(user='pcmanus', game='Coup', year=2015, address=OrderedMapSerializedKey([('home', 'Warsaw'), ('work', 'Piaseczno')]), detailed_scores=OrderedMapSerializedKey([('game1', [1, 2, 3, 4]), ('game2', [40, 50, 60])]), detailed_scores3=OrderedMapSerializedKey([('game1', SortedSet([1, 2, 3, 4])), ('game2', SortedSet([40, 50, 60]))]), score=3000)
Row(user='pcmanus', game='Coup', year=2020, address=None, detailed_scores=None, detailed_scores3=None, score=3000)
Row(user='pcmanus', game='Coup2020', year=2020, address=None, detailed_scores=None, detailed_scores3=None, score=3000)
Row(user='tjake', game='Coup', year=2015, address=None, detailed_scores=None, detailed_scores3=None, score=1000)
Row(user='yukim', game='Coup', year=2015, address=None, detailed_scores=None, detailed_scores3=None, score=2250)
Row(user='jbellis', game='Coup', year=2015, address=None, detailed_scores=None, detailed_scores3=None, score=1750)
TOOK 11.552810668945312 ms


In [28]:
players = [
    'pcmanus', 'tjake', 'yukim', 'jbellis'
]
start = time()

def data(result):
    for row in result:
        print(row)
        
def errback(err):
    print(err)

for player in players:
    print("Execute", player)
    future = session.execute_async("SELECT * FROM scores WHERE user=%s", [player])
    future.add_callbacks(data, errback)
took = time() - start
print("TOOK", took * 1000, "ms")

Execute pcmanus
Execute tjake
Execute yukim
Execute jbellis
TOOK 1.375436782836914 ms
Row(user='pcmanus', game='Coup', year=2015, address=OrderedMapSerializedKey([('home', 'Warsaw'), ('work', 'Piaseczno')]), detailed_scores=OrderedMapSerializedKey([('game1', [1, 2, 3, 4]), ('game2', [40, 50, 60])]), detailed_scores3=OrderedMapSerializedKey([('game1', SortedSet([1, 2, 3, 4])), ('game2', SortedSet([40, 50, 60]))]), score=3000)
Row(user='pcmanus', game='Coup', year=2020, address=None, detailed_scores=None, detailed_scores3=None, score=3000)
Row(user='pcmanus', game='Coup2020', year=2020, address=None, detailed_scores=None, detailed_scores3=None, score=3000)
Row(user='yukim', game='Coup', year=2015, address=None, detailed_scores=None, detailed_scores3=None, score=2250)
Row(user='jbellis', game='Coup', year=2015, address=None, detailed_scores=None, detailed_scores3=None, score=1750)
Row(user='tjake', game='Coup', year=2015, address=None, detailed_scores=None, detailed_scores3=None, score=10

In [30]:
players = [
    'pcmanus', 'tjake', 'yukim', 'jbellis'
]
start = time()

def data(result):
    took = time() - start
    print("GOT", took * 1000, "ms")
        
def errback(err):
    print(err)

for player in players:
    future = session.execute_async("SELECT * FROM scores WHERE user=%s", [player])
    future.add_callbacks(data, errback)
took = time() - start
print("TOOK", took * 1000, "ms")

TOOK 0.6361007690429688 ms
GOT 4.67371940612793 ms
GOT 6.336688995361328 ms
GOT 7.139444351196289 ms
GOT 8.101463317871094 ms
